### First, import the necessary libraries

In [1]:
import keras.utils.vis_utils
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns
import csv
import os
import pandas as pd

In [2]:
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_training-set.csv', header=0)
X_test = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_testing-set.csv', header=0)

X_train = X_train.dropna()
X_test=  X_test.dropna()

Separate labels from data

In [3]:
y_train = X_train.iloc[:,-1]
X_train = X_train.iloc[:,:-2]

y_test = X_test.iloc[:,-1]
X_test = X_test.iloc[:,:-2]

In [4]:
# categorical
for col in ['proto', 'service', 'state']:
    X_train[col] = pd.Categorical(X_train[col]).codes
    X_test[col] = pd.Categorical(X_test[col]).codes
#
# for col in ['proto', 'service', 'state']:
#     X_train[col] = pd.Categorical(X_train[col])
#     X_test[col] = pd.Categorical(X_test[col])
X_train.pop('id')
X_test.pop('id')

0              1
1              2
2              3
3              4
4              5
           ...  
175336    175337
175337    175338
175338    175339
175339    175340
175340    175341
Name: id, Length: 175341, dtype: int64

## Scale the data (BIG, BIG PERFORMANCE INCREASE!!!)

In [5]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

## what if I just threw a NN at it

In [84]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
# define the keras model
batch_size = 128
model = tf.keras.Sequential()
model.add(Dense(42, input_dim=42, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [85]:
from tensorflow.keras.optimizers import Adam
optim = Adam(learning_rate=0.001)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = optim, metrics=['accuracy'])

In [86]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values))
train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values))
test_dataset = test_dataset.shuffle(len(X_test)).batch(batch_size)

In [87]:
history = model.fit(train_dataset, epochs=200, batch_size=batch_size, validation_data=test_dataset)

Epoch 1/200
644/644 [==============================] - 3s 5ms/step - loss: 0.2962 - accuracy: 0.8547 - val_loss: 0.5218 - val_accuracy: 0.7468

KeyboardInterrupt: 

In [88]:
model.save('dropout40pct_one_more_layer.h5')

In [10]:
good_model = tf.keras.models.load_model('dropout30pct.h5')

In [11]:
good_model.evaluate(test_dataset, batch_size=batch_size)

1370/1370 [==============================] - 2s 1ms/step - loss: 0.7408 - accuracy: 0.8871

[0.7407650947570801, 0.8870771527290344]

In [89]:
y_pred = model.predict(X_test)

In [90]:
accuracy = tf.keras.metrics.Precision()
accuracy.update_state(y_test, y_pred)
accuracy.result().numpy()

0.9853397

In [91]:
accuracy = tf.keras.metrics.Recall()
accuracy.update_state(y_test, y_pred)
accuracy.result().numpy()

0.8526659

In [92]:
accuracy = tf.keras.metrics.BinaryAccuracy()
accuracy.update_state(y_test, y_pred)
accuracy.result().numpy()

0.8910865

In [93]:
actual_prediction = np.round(y_pred)

# actually normal, predicted normal
normal_traffic_indices = np.argwhere(y_test.to_numpy()==0)
normal_traffic_indices = normal_traffic_indices.reshape((len(normal_traffic_indices),))
attack_traffic_indices = np.argwhere(y_test.to_numpy()==1)
attack_traffic_indices = attack_traffic_indices.reshape((len(attack_traffic_indices),))

In [94]:
print(f'classified {len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==0)[0])} out of {len(normal_traffic_indices)} normal traffic patterns as normal')

classified 54486 out of 56000 normal traffic patterns as normal


In [95]:
print(f'classified {len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==0)[0])} out of {len(attack_traffic_indices)} attack traffic patterns as normal')

classified 17583 out of 119341 attack traffic patterns as normal


In [96]:
print(f'classified {len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==1)[0])} out of {len(normal_traffic_indices)} normal traffic patterns as attack')

classified 1514 out of 56000 normal traffic patterns as attack


In [97]:
print(f'classified {len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==1)[0])} out of {len(attack_traffic_indices)} attack traffic patterns as attack')

classified 101758 out of 119341 attack traffic patterns as attack
